In [11]:
import pandas as pd
import numpy as np
import seaborn as sn
# Importing Modules
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

/home/activus/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# DL & Unzip data

In [21]:
# !kaggle competitions download -c tabular-playground-series-jul-2022

In [22]:
# !mkdir data

In [23]:
# !unzip tabular-playground-series-jul-2022.zip -d data/

In [24]:
# !rm tabular-playground-series-jul-2022.zip

# Data exploration

In [12]:
df = pd.read_csv('data/data.csv', index_col='id')
df

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28
id,,,,,,,,,,,,,,,,,,,,,
0,-0.389420,-0.912791,0.648951,0.589045,-0.830817,0.733624,2.258560,2,13,14,...,-0.478412,-0.757002,-0.763635,-1.090369,1.142641,-0.884274,1.137896,1.309073,1.463002,0.813527
1,-0.689249,-0.453954,0.654175,0.995248,-1.653020,0.863810,-0.090651,2,3,6,...,-0.428791,-0.089908,-1.784204,-0.839474,0.459685,1.759412,-0.275422,-0.852168,0.562457,-2.680541
2,0.809079,0.324568,-1.170602,-0.624491,0.105448,0.783948,1.988301,5,11,5,...,-0.413534,-1.602377,1.190984,3.267116,-0.088322,-2.168635,-0.974989,1.335763,-1.110655,-3.630723
3,-0.500923,0.229049,0.264109,0.231520,0.415012,-1.221269,0.138850,6,2,13,...,0.619283,1.287801,0.532837,1.036631,-2.041828,1.440490,-1.900191,-0.630771,-0.050641,0.238333
4,-0.671268,-1.039533,-0.270155,-1.830264,-0.290108,-1.852809,0.781898,8,7,5,...,-1.628830,-0.434948,0.322505,0.284326,-2.438365,1.473930,-1.044684,1.602686,-0.405263,-1.987263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97995,0.237591,1.657034,-0.689282,0.313710,-0.299039,0.329139,1.607378,5,7,8,...,-0.290116,-0.258141,-0.973640,1.369508,0.391055,2.152426,-0.208944,-1.475403,0.298448,0.445039
97996,0.322696,0.710411,0.562625,-1.321713,-0.357708,0.182024,0.178558,3,9,2,...,0.117687,1.388242,0.342400,1.680537,-0.860409,0.579165,1.162692,0.134994,0.994666,0.727642
97997,-0.249364,-0.459545,1.886122,-1.340310,0.195029,-0.559520,-0.379767,8,9,10,...,-0.850223,-1.787648,-1.268115,-1.508330,1.945622,1.503645,0.194968,2.142693,1.646042,0.641466


In [13]:
std = StandardScaler()
df = std.fit_transform(df)

test = torch.tensor(df)

In [14]:
tens = torch.tensor(df, dtype=torch.float32)
dataset_train = TensorDataset(tens)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)


In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


/home/activus/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [21]:
import torch.nn.functional as F
n_feature = 29 
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(n_feature, n_feature*2)
        self.bn1 = nn.BatchNorm1d(n_feature*2)
        self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        
        self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        # self.fc4 = nn.Linear(n_feature*8, n_feature*16)
        # self.fc5 = nn.Linear(n_feature*16, n_feature*32)
        # self.fc6 = nn.Linear(n_feature*32, n_feature*64)
        # self.fc7 = nn.Linear(n_feature*64, n_feature*128)
        self.bn2 = nn.BatchNorm1d(n_feature*8)
        self.fc15 = nn.Linear(n_feature*8, 7)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        # x = F.relu(self.fc5(x))
        # x = F.relu(self.fc6(x))
        # x = F.relu(self.fc7(x))
        x = self.bn2(x)
        x = F.relu(self.fc15(x))
                
        return x

In [22]:
model = NeuralNetwork().to(device)
# model.apply(init_weights)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=29, out_features=58, bias=True)
  (bn1): BatchNorm1d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=58, out_features=116, bias=True)
  (fc3): Linear(in_features=116, out_features=232, bias=True)
  (bn2): BatchNorm1d(232, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc15): Linear(in_features=232, out_features=7, bias=True)
)


In [24]:
learning_rate = 0.1
batch_size = 128
epochs = 300
g = 0.985

In [28]:
enumerate(loader_train)

In [40]:
def get_unsupervised_weights(X, n_hidden, n_epochs, batch_size, 
        learning_rate=2e-2, precision=1e-30, anti_hebbian_learning_strength=0.4, lebesgue_norm=2.0, rank=2):
    sample_sz = X.shape[1]    
    weights = torch.rand((n_hidden, sample_sz), dtype=torch.float)  
    for epoch in range(n_epochs):    
        eps = learning_rate * (1 - epoch / n_epochs)        
        shuffled_epoch_data = X[torch.randperm(X.shape[0]),:]
        for i in range(X.shape[0] // batch_size):
            mini_batch = shuffled_epoch_data[i*batch_size:(i+1)*batch_size,:]           
            mini_batch = torch.transpose(torch.tensor(mini_batch, dtype=torch.float32), 0, 1)            
            sign = torch.sign(weights)            
            W = sign * torch.abs(weights) ** (lebesgue_norm - 1)        
            tot_input=torch.mm(W, mini_batch)            
            
            y = torch.argsort(tot_input, dim=0)            
            yl = torch.zeros((n_hidden, batch_size), dtype = torch.float)
            yl[y[n_hidden-1,:], torch.arange(batch_size)] = 1.0
            yl[y[n_hidden-rank], torch.arange(batch_size)] =- anti_hebbian_learning_strength            
                    
            xx = torch.sum(yl * tot_input,1)            
            xx = xx.unsqueeze(1)                    
            xx = xx.repeat(1, sample_sz)                            
            ds = torch.mm(yl, torch.transpose(mini_batch, 0, 1)) - xx * weights            
            
            nc = torch.max(torch.abs(ds))            
            if nc < precision: nc = precision            
            weights += eps*(ds/nc)
            print(nc)
            print(precision)
    return weights

In [51]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# Rudimentary outline of training loop
for epoch in range(20):
    for inputs in loader_train:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, inputs)
        loss.backward()
        optimizer.step()

TypeError: linear(): argument 'input' (position 1) must be Tensor, not list